In [20]:
from pathlib import Path
import czifile
import tifffile
import napari
import pyclesperanto_prototype as cle
import numpy as np
import pandas as pd
from utils import check_filenames, segment_nuclei_2d, segment_marker_positive_nuclei

In [15]:
# Define the intensity threshold above which a cell is considered positive for a marker
neun_channel_threshold = 30
reelin_channel_threshold = 40
gad67_channel_threshold = 40

# Sets the amount of erosion that is applied to areas where the marker+ signal colocalizes with nuclear signal
# The higher the value, the stricter the conditions to consider a nuclei as marker+
neun_erosion_factor = 3
reelin_erosion_factor = 3
gad67_erosion_factor = 3

In [ ]:
# Copy the path where your images are stored, ideally inside the raw_data directory
directory_path = Path("./raw_data/Reelin")
roi_directory_path = Path("./raw_data/Reelin/ROI")

# Define the subdirectories containing your data
subdirectories = ["Contra", "Ipsi", "Sham"]

# Create empty lists to store all image filepaths and ROIs within the dataset directory
images = []
rois = []

# Create an empty list to store all stats extracted from each image
stats = []

# Scan subdirectories and add paths to images fitting certain conditions
for subdir in subdirectories:
    # Construct the subdirectory path
    image_path = directory_path / subdir
    # Iterate through the .czi files in the subdirectories
    for file_path in image_path.glob("*.czi"):
        # Remove unwanted images
        if "AWT" not in str(file_path) and "BWT" not in str(file_path):
            images.append(str(file_path))

# Scan ROI directory and add paths to the list
for file_path in roi_directory_path.glob("*.tif"):
    # Remove unwanted images
        if "AWT" not in str(file_path) and "BWT" not in str(file_path):
            rois.append(str(file_path))

# Check if there is any missing ROI or image file in their corresponding directories
check_filenames(images, rois) 

In [ ]:
for image_path, roi_path in zip(images, rois):
    #TODO: Include the full loop here
    print (image_path)
    print(roi_path)         

In [ ]:
image_path = images[0]
roi_path = rois[0]

# Read image and ROI files into Numpy arrays
img = czifile.imread(image_path)
roi = tifffile.imread(roi_path)

# Remove singleton dimensions and perform MIP on input image
img = img.squeeze()
img_mip = np.max(img, axis=1)

# Perform MIP for the region of interest
roi_mip = np.max(roi, axis=0)

# We will create a mask where label_mip is greater than or equal to 1
mask = roi_mip >= 1

# Apply the mask to img_mip
masked_img = np.where(mask, img_mip, 0)

# Extract each of the channels separately
neun_mip = masked_img[0, :, :]
reelin_mip = masked_img[1, :, :]
gad67_mip = masked_img[2, :, :]
nuclei_mip = masked_img[3, :, :]

# Segment nuclei inside the ROI
nuclei_labels = segment_nuclei_2d(nuclei_mip)

# Dilate or erode nuclei to check for cytoplasmic or nuclear marker colocalization
cyto_nuclei_labels = cle.dilate_labels(nuclei_labels, radius=2)
cyto_nuclei_labels = cle.pull(cyto_nuclei_labels)
eroded_nuclei_labels = cle.erode_labels(nuclei_labels, radius=2)
eroded_nuclei_labels = cle.pull(eroded_nuclei_labels)

# Select marker positive nuclei
neun_tuple = segment_marker_positive_nuclei (nuclei_labels, neun_mip, neun_channel_threshold, neun_erosion_factor)
reelin_tuple = segment_marker_positive_nuclei (cyto_nuclei_labels, reelin_mip, reelin_channel_threshold, reelin_erosion_factor)
gad67_tuple = segment_marker_positive_nuclei (cyto_nuclei_labels, gad67_mip, gad67_channel_threshold, gad67_erosion_factor)


In [ ]:
viewer = napari.Viewer(ndisplay=2)
viewer.add_image(img_mip)
viewer.add_image(roi_mip)
viewer.add_image(masked_img)
viewer.add_labels(nuclei_labels)
viewer.add_labels(cyto_nuclei_labels)
viewer.add_labels(eroded_nuclei_labels)
viewer.add_labels(neun_tuple[1], name="neun_+_nuclei")
viewer.add_labels(reelin_tuple[1], name="reelin_+_nuclei")
viewer.add_labels(gad67_tuple[1], name="gad67_+_nuclei")